# Assignment#2- Hidden Markov Model 
Submitted by:
Akshay Sanjeev. 


A version of this is uploaded to : https://github.com/askay-sanjeev/seqanalysis-homeworks/tree/main/HW-2


In [2]:
import numpy as np
import hw2_fns as HMM
from numpy.random import choice as choose

In [2]:
# from the question
S = ['H', 'T']
T = 3
a = np.array([[  0,  .5,  .5], #transition probabilities
              [.01, .94, .05],
              [.01, .05, .94]])
e = np.array([[.5, .5], #emission probabilities
              [.8, .2]])



## 1. HMM Generator

In [3]:
# Example 
pi,x = HMM.hmm_gen(S, T, a, e)
print(f'length of generated sequence = {len(x)}')
print(f'Hidden state seq: \n {pi}')
print(f'Observable state seq: \n {x}')

length of generated sequence = 319
Hidden state seq: 
 [2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,

## 2. Viterbi Algorithm

In [4]:
path = HMM.Viterbi(x, S, T, a, e)
matchfrac = HMM.match(path, pi)
print(f'{matchfrac*100:.2f}% of the inferred seq matches with the original hidden state sequence.')


41.82% of the inferred seq matches with the original hidden state sequence.


## 3. Forward

In [5]:
f_lk = HMM.Forward(x, S, T, a, e)
print(f'The likelihood of input sequence = {f_lk}') 

The likelihood of input sequence = 4.330785262078143e-87


## 4. Backward 

In [6]:
b_lk = HMM.Backward(x, S, T, a, e)
print(f'The likelihood of input sequence = {b_lk}') 

The likelihood of input sequence = 4.3307852620781525e-87


## 5. Posterior Probability

In [7]:
post = HMM.ForwardBackward(x, S, T, a, e)

#specify the state at which you want to compute the posterior probability
i = 10
print(f'The posterior probabilities of hidden states at {i}:\n p(Fair) = {post[i][0]:.3f} \n p(Biased) = {post[i][1]:.3f}') 

The posterior probabilities of hidden states at 10:
 p(Fair) = 0.131 
 p(Biased) = 0.869
